In [95]:
# open and standardize image files
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from sklearn.cluster import KMeans

#dirs = ['plicatum_36','sargentii_17']
dirs = ['sargentii_17']

def gauss(x,ctr,amp,wid):
    return amp * np.exp( -((x - ctr)/wid)**2)

for d in dirs:
    files = []
    for file in glob.glob(d+'/*.ti*'):
        files.append(file)
    
    # open first image to get the image dimensions
    im = np.array(Image.open(files[0]))
    
    # define image_stack array
    image_stack = np.zeros([len(files),np.shape(im)[0],np.shape(im)[1]])
    # read and standardize all images
    for i in range(len(files)):
        im = np.array(Image.open(files[i]))        
        
        mean = np.mean(im)
        std = np.std(im)
        image_stack[i,:,:] = (im - mean)/std
        
        # for every nth images, explore
        if i%100 == 0:
            indx = files[i].index('.t')
            
            # histogram
            hist, bin_edges = np.histogram(im,bins=1000,range=(0,50000))
            # to avoid log(0) later
            hist[hist < 1] == 0.1
            bins = (bin_edges[1:]+bin_edges[:-1])/2e0            
            
            plt.close()
            width = (bin_edges[1] - bin_edges[0])
            center = (bin_edges[:-1] + bin_edges[1:]) / 2
            plt.xlabel('pixel intensity')
            plt.ylabel('log(count)')
            plt.xlim([0,50000])
            plt.ylim([0,5])
            plt.bar(bin_edges[:-1], np.log10(hist), width)
            plt.tight_layout(w_pad=0, h_pad=0)
            plt.savefig('imgs/histo_'+d+'_'+files[i][indx-4:indx]+'.png')
            
            # LBP
            plt.close()
            lbp = local_binary_pattern(im, 8, 1)
            plt.imshow(lbp)
            plt.savefig('imgs/lbp_'+d+'_'+files[i][indx-4:indx]+'.png')

            # k means clustering
            clustered_img = KMeans(2).fit_predict(im.reshape([np.prod(np.shape(im)),1]))            
            
            clustered_img[clustered_img == clustered_img[0]] = 2e0
            clustered_img[clustered_img != clustered_img[0]] = 3e0
            
            plt.imshow(clustered_img.reshape(np.shape(im)))
            plt.savefig('imgs/kmeans_'+d+'_'+files[i][indx-4:indx]+'.png')
            


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:50: RuntimeWarning: divide by zero encountered in log10
